// Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
// SPDX-License-Identifier: MIT-0

# Create a short text clustering system using AWS SageMaker jumpstart pre-trained transformer models 

1. [Introduction](#Introduction)  
2. [Setup](#Setup)
3. [Create a model for text embeddings from the Jumpstart solutions library of models](#Create-a-model-for-text-embeddings-from-the-Jumpstart-solutions-library-of-models)
4. [Data pre-processing](#Data-pre-processing)
5. [Create phrase (sentence) embeddings](#Create-phrase-(sentence)-embeddings)
6. [Cluster phrases (sentences)](#Cluster-phrases-(sentences))
7. [Automatic cluster labeling](#Automatic-cluster-labeling)
8. [Batch process the entire dataset](#Batch-process-the-entire-dataset)

## Introduction

In this notebook we demonstrate how you can cluster short text (phrases) using the pre-trained transformer models on [AWS SageMaker Jumpstart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html). Here we will demonstrate the use of a transformer model called [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli). The model is used to create an embedding of phrases that we will then use to cluster such phrases.

## Setup

Let's start by updating the required packages i.e. SageMaker Python SDK, pandas, numpy, etc.

In [ ]:
!pip install fasttext-wheel wikipedia boto3 jsonlines seaborn

# **Note: Restart the notebook's kernel after installing the above packages.**

In [ ]:
import boto3
import sagemaker
import json
import re
import os

from sagemaker import get_execution_role

import pandas as pd
import numpy as np
import math

import nltk
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.manifold import TSNE
from sklearn.cluster import SpectralClustering

We use NLTK library to help us with the pre-processing of the data

In [ ]:
session = boto3.Session()
sagemaker_execution_role = get_execution_role()
s3 = session.resource('s3')

In [ ]:
nltk.download('stopwords')

## Create a model for text embeddings from the Jumpstart solutions library of models

We will use one the text embedding [models available in SageMaker jumpstart](https://sagemaker.readthedocs.io/en/v2.129.0/doc_utils/pretrainedmodels.html)

#### Chose a model for Inference

In [ ]:
#We choose the tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1 as the default model since it is better suited for phrase analysis

model_id, model_version = (
    "tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1", 
    "*")

You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).

In [ ]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models, list_jumpstart_tasks

# Retrieves all text embedding models.
filter_value = "task == tcembedding"
tcembedding_models = list_jumpstart_models(filter=filter_value)

# display the model-ids in a dropdown to select a model for inference.
model_dropdown = Dropdown(
    options=tcembedding_models,
    value=model_id,
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [ ]:
display(model_dropdown)

In [ ]:
# model_version="*" fetches the latest version of the model
model_id, model_version = model_dropdown.value, "*"

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"jumpstart-example-infer-{model_id}")

### Create the model from the selected model_id, model_version

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor

inference_instance_type = "ml.m5.xlarge"  #You can change the instance according to your needs

# Retrieve the inference docker container uri. This is the base Tensorflow container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)


# Retrieve the model uri. This includes the model and model parameters.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)


# Create the SageMaker model instance
embedding_model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=model_uri,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    role=sagemaker_execution_role,
    predictor_cls=Predictor,
    name=model_name,
)

## Data pre-processing

For this demonstration we will use a dataset made up of the blog titles for each blog published by AWS from 2004 until late 2022. We use the blog titles to cluster them and assign a topic to each cluster

The text is pre-processed with the following steps:

* Set category string to lowercase
* Replace acronyms with actual words -> the SBW corpus is less likely to have the acronyms in it than the actual words in relation to each other
* Replace special word-bound characters such as / and - (i.e.: imagenes/videos, cerveza-vino) to get separate words.
* Eliminate explanations between parenthesis
* Remove any other non-word characters from sentence
* Split sentence into tokens
* Singularize each token

In [ ]:
def replace_acronyms(sentence):
    for rule in ac:
        match = re.search(rule[0], sentence, re.IGNORECASE)
        if match:
            sentence = re.sub(rule[0], rule[1], sentence)
    
    return sentence

In [ ]:
blogs_df = pd.read_csv('aws_blog_titles.csv', header=None, names=['URL', 'Title'])

aws_acronyms_df = pd.read_csv('acronyms.csv', header=None, delimiter=';', names=['acronym', 'meaning'])

In [ ]:
blogs_df

In [ ]:
#We transform acronyms to their actual meaning since the transformer may not be aware of them (as it was not trained in this specific vocabulary)

aws_acronyms_df

In [ ]:
blogs_df = blogs_df.drop_duplicates(subset=['Title']).reset_index()
blogs_df = blogs_df.drop(columns=['index', 'URL'])

In [ ]:
#For eficiency only take sample_size titles at random

sample_size = 1000
blogs_df_sample = blogs_df.sample(n=sample_size)

In [ ]:
titles = blogs_df_sample['Title'].tolist()

In [ ]:
lemmatized = []
for title in titles:
    sentence = title.lower()
    sentence = re.sub(r'[^a-zA-Z0-9_-áéíóúñ ]', r'', sentence)  #remove extraneous characters (maybe a different encoding)
    lemmatized.append(sentence)

In [ ]:
lemmatized[:10]

## Create phrase (sentence) embeddings

In [ ]:
# These functions are used to query the endpoint and parse the response

def query(model_predictor, text):
    """Query the model predictor."""

    encoded_text = json.dumps(text).encode("utf-8")

    query_response = model_predictor.predict(
        encoded_text,
        {
            "ContentType": "application/x-text",
            "Accept": "application/json",
        },
    )
    return query_response


def parse_response(query_response):
    """Parse response and return the embedding."""

    model_predictions = json.loads(query_response)
    embedding = model_predictions["embedding"]
    return embedding

### Deploy the selected model to an endpoint for real time inference

In [ ]:
# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model_predictor = embedding_model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=model_name,
)

### Generate embeddings

We use the deployed model to generate the embeddings for each of the titles in our sample dataset

In [ ]:
#model_predictor = Predictor('jumpstart-example-infer-tensorflow-tcem-2023-01-19-23-23-44-619')  #Specifiy endpoint name in case you wanna use an already deployed endpoint

In [ ]:
%%time
sentence_vectors = [parse_response(query(model_predictor, title)) for title in lemmatized]

In [ ]:
encoded_titles_df = pd.DataFrame(sentence_vectors)
encoded_titles_df['blog_title_lemmatized'] = lemmatized

In [ ]:
encoded_titles_df

## Cluster phrases (sentences)

Spectral clsutering is a clustering algorithm based on graph theory. Spectral clustering uses information from the eigenvalues (spectrum) of the Laplacian matrix built from the graph or the data set to create groups (clusters) of data. Spectral clustering requires a measures of affinity between data points, for this application we use cosine affinity because we are interested in sentences that lie near to each other but also with "similar meaning".

In [ ]:
n_clusters = 20

clustering_model = SpectralClustering(n_clusters=n_clusters, n_init=100, affinity='cosine', n_neighbors=10, assign_labels="kmeans", random_state=0)
embeddings = encoded_titles_df[encoded_titles_df.columns[0:-1]]
encoded_titles_df['cluster'] = clustering_model.fit_predict(embeddings)

In [ ]:
clusters_titles = encoded_titles_df[['blog_title_lemmatized', 'cluster']]

In [ ]:
clusters_titles

## Automatic cluster labeling

### Use TF-IDF for finding the keywords in each of our clusters

Text Frequency - Inverse Document Frequency is an NLP technique used to find the most relevant terms in set of documents (phrases in our case). From each cluster we extract its most relevant terms (nouns only) according to TF-IDF and use those as labels/categories for that cluster

In [ ]:
clusters = [clusters_titles.loc[clusters_titles.cluster == i, 'blog_title_lemmatized'].to_list() for i in range(0,n_clusters)]

In [ ]:
clusters_tf_idf = []
clusters_tf_idf_terms = []
clusters_tags = []
clusters_keywords_tf_idf = []
tf_idf_threshold = 0.2

for cluster in clusters:

    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cluster)
    tf_idf_df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    tf_idf_df = tf_idf_df.sort_values('TF-IDF', ascending=False)
    
    clusters_tf_idf.append(tf_idf_df)
    
    cluster_tf_idf_terms = list(tf_idf_df.loc[tf_idf_df['TF-IDF'] > tf_idf_threshold].index.values)
    clusters_tf_idf_terms.append(cluster_tf_idf_terms)
    
    tags = nltk.pos_tag(cluster_tf_idf_terms)
    clusters_tags.append(tags)
    
    keywords = [tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    clusters_keywords_tf_idf.append(keywords)

In [ ]:
clusters_idf = []

for cluster in clusters:

    cv = CountVectorizer() 
    word_count_vector = cv.fit_transform(cluster)

    tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
    tfidf_transformer.fit(word_count_vector)

    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
    df_idf['word'] = cv.get_feature_names()

    df_idf = df_idf.sort_values(by='idf_weights')
    clusters_idf.append(df_idf)

In [ ]:
clusters_keywords_tf_idf

In [ ]:
clusters_idf[0][:20]

In [ ]:
clusters_keywords_idf = []
stop_words = stopwords.words('english')

clusters_idf_words = [cluster['word'].to_list()[0:10] for cluster in clusters_idf]
s = [ word for word in stop_words if word != 're'] #Remove stopwords but the word re (for re: invent)

for cluster in clusters_idf_words:
    tags = nltk.pos_tag(cluster)
    words = [ tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    
    clusters_keywords_idf.append(",".join(words))

In [ ]:
clusters_keywords_idf

## Batch process the entire dataset

In this section we will create batch processing jobs to process the entire dataset (roughly 24K titles)

In [ ]:
import io
import jsonlines

from sagemaker.s3 import S3Downloader,S3Uploader,s3_path_join

n_clusters = 20

In [ ]:
bucket_name = '<REPLACE_WITH_YOUR_BUCKET_NAME>'
s3_prefix = '<REPLACE_WITH_YOUR_PREFIX>'

#### Chose a model for Inference

In [ ]:
#We choose the tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1 as the default model since it is better suited for phrase analysis

model_id, model_version = (
    "tensorflow-tcembedding-universal-sentence-encoder-cmlm-en-large-1", 
    "*")

You can continue with the default model, or can choose a different model from the dropdown generated upon running the next cell. A complete list of SageMaker pre-trained models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).

### Create the model from the selected model_id, model_version

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"jumpstart-example-infer-gpu-{model_id}")

In [ ]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor

batch_transform_instance_type = "ml.g4dn.xlarge"

# Retrieve the inference docker container uri. This is the base Tensorflow container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=batch_transform_instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)


# Retrieve the model uri. This includes the model and model parameters.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)


# Create the SageMaker model instance
batch_transform_embedding_model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=model_uri,
    entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
    role=sagemaker_execution_role,
    name=model_name,
)

### Data preprocessing

In [ ]:
blogs_df = pd.read_csv('aws_blog_titles.csv', header=None, names=['URL', 'Title'])
aws_acronyms_df = pd.read_csv('acronyms.csv', header=None, delimiter=';', names=['acronym', 'meaning'])
blogs_df = blogs_df.drop_duplicates(subset=['Title']).reset_index()
blogs_df = blogs_df.drop(columns=['index', 'URL'])
titles = blogs_df['Title'].tolist()

In [ ]:
lemmatized = []
for title in titles:
    sentence = title.lower()
    sentence = re.sub(r'[^a-zA-Z0-9_-áéíóúñ ]', r'', sentence)  #remove extraneous characters (maybe a different encoding)
    lemmatized.append(sentence)

### Upload the pre-processed data to S3

In [ ]:
batch_filename = 'aws_blog_titles.jsonl'

In [ ]:
with open(batch_filename, "wb") as txt_file:
    for title in lemmatized:
        
        txt_file.write(json.dumps(title).encode("utf-8"))
        txt_file.write("\n".encode('utf-8'))

In [ ]:
data_upload_path = s3_path_join("s3://",bucket_name,s3_prefix, 'raw')
print(f"Uploading data to {data_upload_path}")
data_uri = S3Uploader.upload(batch_filename, data_upload_path)
print(f"Uploaded data to {data_upload_path}")

### Generate embeddings

In [ ]:
# create transformer to run a batch job

output_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "embeddings")

batch_job = batch_transform_embedding_model.transformer(
    instance_count=1,
    instance_type=batch_transform_instance_type,
    strategy='SingleRecord',
    assemble_with='Line',
    output_path=output_path,
)

In [ ]:
# Starts batch transform job and uses S3 data as input. Enable the logs and wait only if you pass a small number of samples (< 100).
# You can monitor your batch processing job from the SageMaker Console -> Inference -> Batch transform jobs
batch_job.transform(
    data=data_upload_path,
    content_type='application/x-text',    
    split_type='Line',
    logs=False,
    wait=False
)

In [ ]:
#Download the results. 
#The batch transformation job (step above) must have finished before you can run this cell.
embedding_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "embeddings", batch_filename+'.out')
print(f"Downloading embeddings to .")
S3Downloader.download(embedding_data_path,'.')
print(f"Downloaded embeddings to .")

In [ ]:
lines = []

with jsonlines.open(batch_filename+".out", mode='r') as reader:
    for obj in reader:
        lines.append(obj['embedding'])
        
results_df = pd.DataFrame(lines)
results_df['blog_title_lemmatized'] = lemmatized

In [ ]:
results_df

In [ ]:
embeddings_filename = "blog_title_embeddings.csv"
results_df.to_csv(embeddings_filename, index=False)

In [ ]:
embedding_upload_path = s3_path_join("s3://",bucket_name,s3_prefix, 'embeddings')
print(f"Uploading embeddings to {embedding_upload_path}")
data_uri = S3Uploader.upload(embeddings_filename, embedding_upload_path)
print(f"Uploaded embeddings to {embedding_upload_path}")

### Cluster titles

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

In [ ]:
sklearn_processor_spectral_clustering = SKLearnProcessor(framework_version='1.0-1',
                                                         role=sagemaker_execution_role,
                                                         instance_type='ml.m5.2xlarge',
                                                         instance_count=1)

In [ ]:
output_destination = os.path.join('s3://', bucket_name, s3_prefix, "results", "clusters")

sklearn_processor_spectral_clustering.run(
    code="./scikit-sagemaker-clustering/SpectralClustering.py",
    inputs=[ProcessingInput(source=embedding_upload_path, destination="/opt/ml/processing/input")],
    outputs=[ProcessingOutput(output_name="titles_clusters", source="/opt/ml/processing/output", destination=output_destination)],
    arguments=["--n-clusters", str(n_clusters),
               "--n-init", "100",
               "--affinity", "cosine",
               "--n-neighbors", "10",
               "--assign-labels", "kmeans"
              ],
)

In [ ]:
#Download the results. 
#The batch clustering job (step above) must have finished before you can run this cell.

clusters_file = 'clustered_blog_titles_with_embeddings.csv'

clusters_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "clusters", clusters_file)
print(f"Downloading cluster data to .")
S3Downloader.download(clusters_data_path,'.')
print(f"Downloaded cluster data to .")

### Automatic cluster labeling

In [ ]:
clusters_df = pd.read_csv(clusters_file)
clusters_titles = clusters_df[['blog_title_lemmatized', 'cluster_label']]

In [ ]:
clusters_titles

In [ ]:
clusters = [clusters_titles.loc[clusters_titles.cluster_label == i, 'blog_title_lemmatized'].to_list() for i in range(0, n_clusters)]

In [ ]:
clusters_tf_idf = []
clusters_tf_idf_terms = []
clusters_tags = []
clusters_keywords_tf_idf = []
tf_idf_threshold = 0.2

for cluster in clusters:

    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(cluster)
    tf_idf_df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    tf_idf_df = tf_idf_df.sort_values('TF-IDF', ascending=False)
    
    clusters_tf_idf.append(tf_idf_df)
    
    cluster_tf_idf_terms = list(tf_idf_df.loc[tf_idf_df['TF-IDF'] > tf_idf_threshold].index.values)
    clusters_tf_idf_terms.append(cluster_tf_idf_terms)
    
    tags = nltk.pos_tag(cluster_tf_idf_terms)
    clusters_tags.append(tags)
    
    keywords = [tag[0] for tag in tags if tag[1] in ['NN', 'NNS'] and tag[0] not in ['aws', 'amazon']]
    clusters_keywords_tf_idf.append(keywords)

In [ ]:
clusters_keywords_tf_idf

In [ ]:
clusters_df['categories'] = clusters_titles['cluster_label'].map(lambda i: clusters_keywords_tf_idf[i])

In [ ]:
clusters_df.loc[clusters_df['cluster_label']==0, ['blog_title_lemmatized', 'cluster_label', 'categories']]

In [ ]:
clusters_categories_file = 'aws_blog_titles_clusters_categories.csv'
clusters_df.to_csv(clusters_categories_file, index=False)

In [ ]:
clusters_data_path = s3_path_join("s3://", bucket_name, s3_prefix, "results", "clusters")
print(f"Uploading clusters to {clusters_data_path}")
clusters_file_uri = S3Uploader.upload(clusters_categories_file, clusters_data_path)
print(f"Uploaded clusters to {clusters_data_path}")

### Visualize the clusters

In [ ]:
cluster_sample_df = clusters_df.sample(n=1000).reset_index()
title_embeddings_sample = cluster_sample_df.iloc[:,:-3]
clusters_titles_sample = cluster_sample_df[['blog_title_lemmatized', 'cluster_label', 'categories']]
clusters_titles_sample['short_categories'] = clusters_titles_sample['categories'].map(lambda x: x[:2])

In [ ]:
clusters_titles_sample

In [ ]:
clusters_tsne = TSNE(perplexity=13, n_components=2, init='pca', n_iter=5000)
tsne_embeddings = clusters_tsne.fit_transform(title_embeddings_sample)
tsne_embeddings_df = pd.DataFrame(tsne_embeddings, columns=['x', 'y'])
tsne_embeddings_df['cluster'] = clusters_titles_sample['cluster_label']
tsne_embeddings_df['labels'] = clusters_titles_sample['categories']

In [ ]:
tsne_embeddings_df

In [ ]:
colors=[
    '#efaf50',
    '#a09934',
    '#e31ad9',
    '#cfcbb0',
    '#1224c9',
    '#669fa4',
    '#087274',
    '#787168',
    '#3e93cb',
    '#722823',
    '#c8784c',
    '#74ac48',
    '#c31033',
    '#5acc21',
    '#2ef8ba',
    '#c67ebe',
    '#805004',
    '#a8f43b',
    '#442d6d',
    '#9141ea',
]

fig, ax = plt.subplots(figsize=(30,30))
ax = sns.scatterplot(data=tsne_embeddings_df, x='x', y='y', hue='cluster', legend='full', palette=colors, ax=ax)
plt.show()

In [ ]:
tsne_embeddings_df[['cluster', 'labels']].drop_duplicates('cluster').sort_values('cluster')